In [62]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

import pandas as pd
import glob as glob

len_delta = 38
len_theta = 38
len_alpha = 70
len_beta = 133
len_gamma = 259

idx_delta = 4
idx_theta = 3
idx_alpha = 2
idx_beta = 1
idx_gamma = 0

In [63]:
def layer(W, num_wires):
    for i in range(num_wires):
        qml.Rot(W[i, 0], W[i, 1], W[i, 2], wires=i)
        
    for j in range(num_wires):
        if j != num_wires-1:
            qml.CNOT(wires=[j, j+1])
        else:
            qml.CNOT(j, 0)

In [64]:
def get_raw_angles(x, num_wires):
    angles = []
    
    squared_sum = lambda input_arr: sum([d**2 for d in input_arr])
    
    for k in range(num_wires):
        ang_k = []
        for j in range(2**(num_wires-k-1)):
            num_idx = (2*j+1)*(2**k)
            den_idx = j*(2**(k+1))
            ang = 2 * np.arcsin(np.sqrt(squared_sum(x[num_idx : num_idx+(2**k-1)+1])) / np.sqrt(squared_sum(x[den_idx : den_idx+2**(k+1)]) + 1e-12))
            ang_k.append(ang)
            
        angles.insert(0, ang_k)
            
    return angles

In [65]:
def grayCodeUtil(res, n, num):
    if (n == 0): 
        res.append(num[0]) 
        return
    
    grayCodeUtil(res, n - 1, num) 
    
    num[0] = num[0] ^ (1 << (n - 1))  
    grayCodeUtil(res, n - 1, num)

def generate_gray_sequence(n):
    res = []
    
    num = [0]
    grayCodeUtil(res, n, num)
    return res

def alpha_to_theta(alphas):
    k = int(np.log(len(alphas)) / np.log(2)) 
    thetas = []
    M = np.zeros((2**k, 2**k))
    
    print(alphas)
    
    for i in range(2**k):
        for j in generate_gray_sequence(k):
            M[i][j] = ((-1) ** (i & j)) / (2**k)
            
    print(M)
    
    thetas = M.dot(alphas)
    
    return thetas
    
# alpha_to_theta(np.ones((2, 1)))

In [210]:
# TODO: Implement:-
# 1. Gray code
# 2. Alpha angle to theta angle tranformation (Not sure?)

def get_angles(x):
    num_wires = int(np.log(len(x)) / np.log(2))
    raw_ang = get_raw_angles(x, num_wires)
    angles = []
#     print(raw_ang)
    
#     print(raw_ang[0][0], -raw_ang[1][0]/2, raw_ang[1][0]/2, -raw_ang[1][1]/2, raw_ang[1][1]/2)
    
#     num_wires =  k 
#     k/2
#     np.append(0th one)
#     for i in range((2**(n+1) - 4))/2):
#         np.append(raw_angle[1][i]/2**k)
#         np.append(-raw_angle[1][i]/2**k)
    
    angles.append(raw_ang[0][0])
    
    for i in range(1, len(raw_ang)):
        raw_ang[i].reverse()
        idx = 0
        for j in range(len(raw_ang[i])):
            if j < len(raw_ang[i])//2:
                angles.insert(2*idx, raw_ang[i][j]/2)
                angles.insert(2*idx+1, -raw_ang[i][j]/2)
                idx += 1
            else:
                angles.append(raw_ang[i][j]/2)
                angles.append(-raw_ang[i][j]/2)
            
            
    
    return np.array(angles)



x = np.array([0.53896774, 0.79503606, 0.27826503, 0.0])
# x = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
# x = np.array([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
get_angles(x)

array([ 0.        , -0.        ,  0.56397465,  0.97504604, -0.97504604])

In [211]:
def create_ctrl_rot(num_wires):
    if num_wires == 2:
        return [num_wires-1, num_wires-1]
    else:
        ctrl_idxs = create_ctrl_rot(num_wires-1)
        ctrl_idxs[-1] += 1
        return ctrl_idxs + ctrl_idxs

In [234]:
def UCRy(angles, wires):
#     print('UCRy:')
    num_ctrl_rot = 2 ** len(wires) - 1
    ctrl_idxs = [len(wires)-i-1 for i in create_ctrl_rot(len(wires))]
    
    i = 0
    for idx in ctrl_idxs:
        qml.RY(angles[i], wires=wires[-1])
#         print(f'RY(a[{i}]), wires={wires[-1]})')
        qml.CNOT(wires=[idx, wires[-1]])
#         print(f'CNOT(wires=[{idx}, {wires[-1]}])')
        i += 1
        
#     print()

In [235]:
def XUCRy(angles, wires):
#     print('XUCRy:')
    for i in wires[:-1]:
        qml.PauliX(wires=i)
        
    UCRy(angles, wires)
    
    for i in wires[:-1]:
        qml.PauliX(wires=i)

In [236]:
def statePreparation(a, num_wires):
    idx = 0
#     print('num_wires (SP):', num_wires, 'num_ang:', len(a), 'mid_point:', len(a)//2)
    qml.RY(a[len(a)//2], wires=0)
    for i in range(num_wires):
        if idx == (len(a)//2):
#             print(f'RY(a[{idx}]), wires=0)')
#             print()
            idx += 1
        else:
            wires = [num for num in range(num_wires-i)]
#             print('UCRy: \t i:', i, 'SP wires:', wires, 'idx:', idx)
            UCRy(a[idx : (idx+(2**(num_wires-i)))], wires)
            idx += (2**(num_wires-i-1))
#             print('idx_later:', idx)

    
    for i in range(num_wires-1, 0, -1):
        wires = [num for num in range(num_wires-i+1)]
#         print('XUCRy: \t i:', i, 'SP wires:', wires, 'idx:', idx)
        XUCRy(a[idx : (idx+(2**(num_wires-i)))], wires)
        idx += (2**(num_wires-i))
#         print('idx_later:', idx)

In [237]:
def normalize(list_data):
    for i in range(2):
        mu = list_data[i].mean()
        sigma = list_data[i].std()
        
        list_data[i] = (list_data[i] - mu) / sigma
        for j in range(len(list_data[i])):
            if list_data[i][j] > 2:
                list_data[i][j] = 2
            elif list_data[i][j] < -2:
                list_data[i][j] = -2
                
        list_data[i] = (list_data[i] / 5) + 0.5
            
    return list_data

In [239]:
# files = glob.glob('./DWT_TextDump/*/*.txt')
# data = pd.read_csv(files[0], delimiter='\t', header=None)

# list_data = [np.asanyarray(data[3])[:32], np.asanyarray(data[4])[:32]]

# norm_data = normalize(list_data)

# x = norm_data[0] # Load theta wave values
# x = np.array([0.53896774, 0.79503606, 0.27826503, 0.0, 0.53896774, 0.79503606, 0.27826503, 0.0])
x = np.array([0.5, 0.5, 0.5, 0.5])#, 0.5, 0.5, 0.5, 0.5])
ang = get_angles(x)

# num_qubits = int(np.log(len(norm_data[0]))/np.log(2))
num_qubits = int(np.log(len(x))/np.log(2))
# num_qubits = 2

dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev)
def test(angles=None):
    statePreparation(angles, num_qubits)
    return qml.expval(qml.PauliZ(0))

test(angles=ang)

print("x               : ", x)
print("angles          : ", ang)
print("amplitude vector: ", np.real(dev._state))

x               :  [0.5 0.5 0.5 0.5]
angles          :  [ 0.78539816 -0.78539816  1.57079633  0.78539816 -0.78539816]
amplitude vector:  [0.5 0.5 0.5 0.5]
